### Reading all VCF data and saving it as CSV files

This notebook is here to convert VCF files to something more familiar and easier to use - CSV files. However, I do not claim to have enough expertise in working with VCF files to say if this makes sense and if I have done it correctly here. As such, I would like to ask other team members to review this code, check if such transformation even makes sense and improve this notebook as they see fit.

In [1]:
!pip install "setuptools<58" --upgrade
!pip uninstall pyvcf
!pip install pyvcf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 57.5.0 which is incompatible.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 57.5.0 which is incompatible.


  Preparing metadata (setup.py) ... done
  Created wheel for pyvcf: filename=PyVCF-0.6.8-cp310-cp310-linux_x86_64.whl size=174265 sha256=6f62a567c25d33757750fddae1d38b530818cdd133b560904fdc38b1de4b4d72
  Stored in directory: /root/.cache/pip/wheels/da/88/3b/b691691894da44fc597c93ba7f953a46d1413ed7411906a6d2
Successfully built pyvcf


In [1]:
file1 = "/content/drive/MyDrive/Pathogenicity_Data/EE_015.vcf.gz"
file2 = "/content/drive/MyDrive/Pathogenicity_Data/EE_050.vcf.gz"
file3 = "/content/drive/MyDrive/Pathogenicity_Data/EE_069.vcf.gz"

In [3]:
import vcf
import pandas as pd

In [30]:
def convert_to_csv_test(vcf_file):
    data = []

    with open(vcf_file, 'r') as vcf_file_binary:
        vcf_reader = vcf.Reader(vcf_file_binary)
        column_headers = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "FORMAT"]

        column_headers.extend(vcf_reader.infos.keys())
        column_headers.append("SAMPLE")

        for record in vcf_reader:
            row = []
            row.extend([record.CHROM, record.POS, record.ID, record.REF, record.ALT, record.QUAL, record.FILTER, record.FORMAT])

            for field in vcf_reader.infos.keys():
                info_value = record.INFO.get(field)
                row.append(info_value)

            for sample in record.samples:
                # tumor sample info in order specified in FORMAT
                row.append("|".join([str(x) for x in sample.data]))

            data.append(row)

    df = pd.DataFrame(data, columns=column_headers)
    return df

convert_to_csv_test("data/EE_sample.vcf")

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,ACMG_class,ACMG_coding_impact,...,gnomadGenomesAC,gnomadGenomesAN,gnomadGenomesEthnic_AC_Hom,gnomadGenomes_AC_Hemi,gnomadGenomes_AC_Hom,gnomadGenomes_AF,gnomadGenomes_AF_ethnic,hgvs,CSQ,SAMPLE
0,chr1,15820,rs2691315,G,[T],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[18059],[82218],[233],None,[18059],[0.219648],[0.0938074],None,[T|downstream_gene_variant|MODIFIER|DDX11L1|EN...,"0/1|0.204545|[35, 9]|0.21|44|[20, 6]|[15, 2]|[..."
1,chr1,17385,rs201535981,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[21384],[111582],[1],None,[21384],[0.191644],[0.213724],None,[A|downstream_gene_variant|MODIFIER|DDX11L1|EN...,"0/1|0.333333|[46, 23]|0.336|69|[28, 10]|[17, 1..."
2,chr1,17697,rs71260069,G,[C],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Benign],[non%40coding],...,[17539],[116050],[2],None,[17539],[0.151133],[0.178196],None,[C|downstream_gene_variant|MODIFIER|DDX11L1|EN...,"0/1|0.170213|[39, 8]|0.176|47|[21, 4]|[17, 3]|..."
3,chr1,133129,rs367730352,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[72154],[131698],[7468],None,[72154],[0.547875],[0.597389],None,[A|non_coding_transcript_exon_variant|MODIFIER...,"0/1|1.0|[0, 20]|0.956|20|[0, 9]|[0, 11]|[0, 0,..."
4,chr1,183629,rs71267774,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Benign],[non%40coding],...,[14902],[111796],[1],None,[14902],[0.133296],[0.220891],None,[A|downstream_gene_variant|MODIFIER|MIR6859-2|...,"0/1|0.242647|[103, 33]|0.246|136|[46, 14]|[55,..."
5,chr1,184267,rs1396210256,C,[T],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Likely%40Benign],[non%40coding],...,[887],[144780],[0],None,[887],[0.00612654],[0.00631038],None,[T|downstream_gene_variant|MODIFIER|MIR6859-2|...,"0/1|0.0972222|[65, 7]|0.108|72|[34, 3]|[27, 4]..."


In [8]:
def convert_to_csv(vcf_file):
    data = []

    with open(vcf_file, 'rb') as vcf_file_binary:
        vcf_reader = vcf.Reader(vcf_file_binary)
        column_headers = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "FORMAT"]

        column_headers.extend(vcf_reader.infos.keys())
        column_headers.append("SAMPLE")

        for record in vcf_reader:
            row = []
            row.extend([record.CHROM, record.POS, record.ID, record.REF, record.ALT, record.QUAL, record.FILTER, record.FORMAT])

            for field in vcf_reader.infos.keys():
                info_value = record.INFO.get(field)
                row.append(info_value)

            for sample in record.samples:
                row.append(sample['GT'])

            data.append(row)

    df = pd.DataFrame(data, columns=column_headers)
    return df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,ACMG_class,ACMG_coding_impact,...,gnomadGenomesAC,gnomadGenomesAN,gnomadGenomesEthnic_AC_Hom,gnomadGenomes_AC_Hemi,gnomadGenomes_AC_Hom,gnomadGenomes_AF,gnomadGenomes_AF_ethnic,hgvs,CSQ,SAMPLE
0,chr1,15820,rs2691315,G,[T],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[18059],[82218],[233],None,[18059],[0.219648],[0.0938074],None,[T|downstream_gene_variant|MODIFIER|DDX11L1|EN...,0/1
1,chr1,17385,rs201535981,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[21384],[111582],[1],None,[21384],[0.191644],[0.213724],None,[A|downstream_gene_variant|MODIFIER|DDX11L1|EN...,0/1
2,chr1,17697,rs71260069,G,[C],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Benign],[non%40coding],...,[17539],[116050],[2],None,[17539],[0.151133],[0.178196],None,[C|downstream_gene_variant|MODIFIER|DDX11L1|EN...,0/1
3,chr1,133129,rs367730352,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[72154],[131698],[7468],None,[72154],[0.547875],[0.597389],None,[A|non_coding_transcript_exon_variant|MODIFIER...,0/1
4,chr1,183629,rs71267774,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Benign],[non%40coding],...,[14902],[111796],[1],None,[14902],[0.133296],[0.220891],None,[A|downstream_gene_variant|MODIFIER|MIR6859-2|...,0/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102512,chrY,56856512,rs76439603,A,[C],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:PGT:PID:PS:SB,[Uncertain%40Significance],[non%40coding],...,None,None,None,None,None,None,None,None,[C|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1
102513,chrY,56856517,rs75739861,A,[G],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:PGT:PID:PS:SB,[Uncertain%40Significance],[non%40coding],...,None,None,None,None,None,None,None,None,[G|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1
102514,chrY,56856554,rs77088808,T,[G],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,None,None,None,None,None,None,None,None,[G|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1
102515,chrY,56856581,rs9320083,G,[A],None,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,[Uncertain%40Significance],[non%40coding],...,[2],[135],[0],None,[2],[0.0148148],[0.0],None,[A|downstream_gene_variant|MODIFIER|CTBP2P1|EN...,0/1


In [ ]:
EE_015=convert_to_csv(file1)
EE_015.to_csv('/content/drive/MyDrive/Pathogenicity_Data/EE_015.csv', index=False)

In [ ]:
EE_050=convert_to_csv(file2)
EE_050.to_csv('/content/drive/MyDrive/Pathogenicity_Data/EE_050.csv', index=False)

In [5]:
EE_069=convert_to_csv(file3)
EE_069.to_csv('/content/drive/MyDrive/Pathogenicity_Data/EE_069.csv', index=False)